# Data pre_processing

In [11]:
import pandas as pd
import nltk
id = pd.read_csv("D:\Kaggle\dm2020-hw2-nthu\data_identification.csv",sep=',', header=0)
train_id = id[id['identification']=='train'].tweet_id
test_id = id[id['identification']=='test'].tweet_id
test_id

0          0x28cc61
3          0x2db41f
15         0x2466f6
23         0x23f9e9
31         0x1fb4e1
             ...   
1867495    0x2c4dc2
1867496    0x31be7c
1867500    0x1ca58e
1867515    0x35c8ba
1867518    0x1d941b
Name: tweet_id, Length: 411972, dtype: object

In [12]:
labeled_emo = pd.read_csv("D:\Kaggle\dm2020-hw2-nthu\emotion.csv",sep=',', header=0)
labeled_emo

,tweet_id,emotion
0,0x3140b1,sadness
1,0x368b73,disgust
2,0x296183,anticipation
3,0x2bd6e1,joy
4,0x2ee1dd,anticipation
...,...,...
1455558,0x38dba0,joy
1455559,0x300ea2,joy
1455560,0x360b99,fear
1455561,0x22eecf,joy


In [13]:
import json
file = "D:/Kaggle/dm2020-hw2-nthu/tweets_DM.json"
with open(file,'r') as r:
    tweets =[]
    for line in r.readlines():
        tweets.append(json.loads(line))
# converting json dataset from dictionary to dataframe
raw = pd.DataFrame.from_dict(tweets)

In [14]:
raw[raw['_index'] != 'hashtag_tweets']

,_score,_index,_source,_crawldate,_type


In [15]:
raw[raw['_type'] != 'tweets']

,_score,_index,_source,_crawldate,_type


In [16]:
dic1 = raw['_source'][1]
dic1['tweet']

{'hashtags': ['freepress', 'TrumpLegacy', 'CNN'],
 'tweet_id': '0x2d5350',
 'text': '@brianklaas As we see, Trump is dangerous to #freepress around the world. What a <LH> <LH> #TrumpLegacy.  #CNN'}

In [17]:
#row['_sorce']= row['_sorce'] lambda x : x['tweet']
hashtage = [ i['tweet']['hashtags'] for i in raw['_source']]
tweet_id = [ i['tweet']['tweet_id'] for i in raw['_source']]
text = [ i['tweet']['text'] for i in raw['_source']]

In [18]:
raw['hashtage']=hashtage
raw['tweet_id']=tweet_id
raw['text']=text

In [19]:
process=raw.drop(['_crawldate','_index','_type','_source'],axis=1)
process

,_score,hashtage,tweet_id,text
0,391,[Snapchat],0x376b20,"People who post ""add me on #Snapchat"" must be ..."
1,433,"[freepress, TrumpLegacy, CNN]",0x2d5350,"@brianklaas As we see, Trump is dangerous to #..."
2,232,[bibleverse],0x28b412,"Confident of your obedience, I write to you, k..."
3,376,[],0x1cd5b0,Now ISSA is stalking Tasha 😂😂😂 <LH>
4,989,[],0x2de201,"""Trust is not the same as faith. A friend is s..."
...,...,...,...,...
1867530,827,"[mixedfeeling, butimTHATperson]",0x316b80,When you buy the last 2 tickets remaining for ...
1867531,368,[],0x29d0cb,I swear all this hard work gone pay off one da...
1867532,498,[],0x2a6a4f,@Parcel2Go no card left when I wasn't in so I ...
1867533,840,[],0x24faed,"Ah, corporate life, where you can date <LH> us..."


In [20]:
train_df =pd.merge(labeled_emo,process,left_on='tweet_id', right_on='tweet_id')
train_df[0:5]

,tweet_id,emotion,_score,hashtage,text
0,0x3140b1,sadness,105,[],Why Chester? <LH> <LH>
1,0x368b73,disgust,797,[],@JaredLeto you are the fish that Jonah. Excep...
2,0x296183,anticipation,586,[pouroutyourspirit],He is coming back again and gonna come again q...
3,0x2bd6e1,joy,780,[],Dei is really such a beautiful person inside &...
4,0x2ee1dd,anticipation,329,[],Expressive praise is also an expression of fai...


In [21]:
test_set =pd.DataFrame(test_id)
test_df = pd.merge(test_set,process,left_on='tweet_id', right_on='tweet_id')
test_df[0:5]

,tweet_id,_score,hashtage,text
0,0x28cc61,107,[],@Habbo I've seen two separate colours of the e...
1,0x2db41f,728,[],@FoxNews @KellyannePolls No serious self respe...
2,0x2466f6,491,[womendrivers],"Looking for a new car, and it says 1 lady owne..."
3,0x23f9e9,28,[robbingmembers],@cineworld “only the brave” just out and fount...
4,0x1fb4e1,925,[],Felt like total dog 💩 going into open gym and ...


# BOW Vectorizer

In [17]:
from sklearn.feature_extraction.text import CountVectorizer

BOW_500 = CountVectorizer(max_features=500, tokenizer=nltk.word_tokenize) 

# apply analyzer to training data
BOW_500.fit(train_df['text'])

train_data_BOW_features_500 = BOW_500.transform(train_df['text'])

## check dimension
train_data_BOW_features_500.shape

(1455563, 500)

In [19]:
X_train = train_data_BOW_features_500
y_train= train_df['emotion']
X_test = BOW_500.transform(test_df['text'])
y_test = ['joy' for i in range(len(test_df['text']))]

# Using LogisticRegression

In [22]:
# Answer here
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
## training!
clf_model = LogisticRegression(random_state=0).fit(X_train, y_train)
## predict!
y_train_pred = clf_model.predict(X_train)
y_test_pred = clf_model.predict(X_test)
acc_train = accuracy_score(y_true=y_train, y_pred=y_train_pred)
acc_test = accuracy_score(y_true=y_test, y_pred=y_test_pred)

In [23]:
print('LR_training accuracy: {}'.format(round(acc_train, 2)))
print('LR_testing accuracy: {}'.format(round(acc_test, 2)))

LR_training accuracy: 0.46
LR_testing accuracy: 0.6


In [28]:
with open('kaggle_demo1.csv','w') as w:
    w.write('id,emotion\n')
    for i,j in zip(test_df['tweet_id'],y_test_pred):
        w.write(i+','+j+'\n')


# DNN Method
## Using Pre-training Glove Word2Vec
## Using Text classification with Transformer

In [22]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [23]:

train_df['token'] = train_df['text'].apply(lambda x: nltk.word_tokenize(x))
test_df['token'] = test_df['text'].apply(lambda x: nltk.word_tokenize(x))
train_df['token'][0:3]


0                [Why, Chester, ?, <, LH, >, <, LH, >]
1    [@, JaredLeto, you, are, the, fish, that, Jona...
2    [He, is, coming, back, again, and, gon, na, co...
Name: token, dtype: object

In [24]:
import numpy as np
# load pretrained word vectors
# get a dict of tokens (key) and their pretrained word vectors (value)
# pretrained word2vec CBOW word vector: https://fgc.stpi.narl.org.tw/activity/videoDetail/4b1141305ddf5522015de5479f4701b1
dim = 0
word_vecs= {}
with open('D:/Word2Vec/glove/glove.twitter.27B.100d.txt') as f:
    for line in f:
        tokens = line.strip().split()

        # there 2 integers in the first line: vocabulary_size, word_vector_dim
        if len(tokens) == 2:
            dim = int(tokens[1])
            continue
    
        word = tokens[0] 
        vec = np.array([ float(t) for t in tokens[1:] ])
        word_vecs[word] = vec
# look up word vectors
# turn each word into its pretrained word vector
# return a list of word vectors corresponding to each token in train.data
def Word2Vector(data_list, embedding_dict):
    embedding_list = list()

    # No Match Word (unknown word) Vector in Embedding
    unk_vector=np.random.rand(*(list(embedding_dict.values())[0].shape))

    for idx_list in range(len(data_list)):
        embedding_list_tmp = list()
        for idx_tuple in range(len(data_list[idx_list])):
            key = data_list[idx_list][idx_tuple][0] # token

            if key in embedding_dict:
                value = embedding_dict[key]
            else:
                value = unk_vector
            embedding_list_tmp.append(value)
        embedding_list.append(embedding_list_tmp)
    return embedding_list

In [25]:

print('Word2Vector(traindata_list, word_vecs)...')
trainembed_list = Word2Vector(train_df['token'], word_vecs)
print('Word2Vector(testdata_list, word_vecs)...')
testembed_list = Word2Vector(test_df['token'], word_vecs)


Word2Vector(traindata_list, word_vecs)...
Word2Vector(testdata_list, word_vecs)...


In [26]:
len(trainembed_list)

1455563

In [27]:
trainembed_list[0]

[array([0.53220808, 0.06090363, 0.45627118, 0.4205117 , 0.13833629,
        0.86486041, 0.92912644, 0.96014248, 0.11320715, 0.660196  ,
        0.33268148, 0.50669442, 0.92907561, 0.4261753 , 0.9083427 ,
        0.90329995, 0.90241052, 0.62823615, 0.8072786 , 0.17145464,
        0.75900622, 0.42987173, 0.08832042, 0.45669124, 0.41205393,
        0.50473491, 0.2236267 , 0.71758394, 0.0425141 , 0.08713762,
        0.61706747, 0.01649812, 0.75161735, 0.02453985, 0.01602739,
        0.74127032, 0.0149108 , 0.49584517, 0.14689757, 0.51235504,
        0.39783266, 0.36120258, 0.97461113, 0.48731412, 0.53210786,
        0.8289734 , 0.10084957, 0.03197739, 0.69071014, 0.12389173,
        0.23238007, 0.84079452, 0.6089615 , 0.62258558, 0.22381935,
        0.70389084, 0.22032252, 0.44575156, 0.57539032, 0.06705461,
        0.18199977, 0.17761068, 0.66142716, 0.75462676, 0.08169912,
        0.45172489, 0.42761314, 0.22384598, 0.41303066, 0.86328424,
        0.68612812, 0.13194276, 0.62372889, 0.44

In [28]:
maxlen = max([len(i) for i in trainembed_list])

In [29]:
class MultiHeadSelfAttention(layers.Layer):
    def __init__(self, embed_dim, num_heads=8):
        super(MultiHeadSelfAttention, self).__init__()
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        if embed_dim % num_heads != 0:
            raise ValueError(
                f"embedding dimension = {embed_dim} should be divisible by number of heads = {num_heads}"
            )
        self.projection_dim = embed_dim // num_heads
        self.query_dense = layers.Dense(embed_dim)
        self.key_dense = layers.Dense(embed_dim)
        self.value_dense = layers.Dense(embed_dim)
        self.combine_heads = layers.Dense(embed_dim)

    def attention(self, query, key, value):
        score = tf.matmul(query, key, transpose_b=True)
        dim_key = tf.cast(tf.shape(key)[-1], tf.float32)
        scaled_score = score / tf.math.sqrt(dim_key)
        weights = tf.nn.softmax(scaled_score, axis=-1)
        output = tf.matmul(weights, value)
        return output, weights

    def separate_heads(self, x, batch_size):
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.projection_dim))
        return tf.transpose(x, perm=[0, 2, 1, 3])

    def call(self, inputs):
        # x.shape = [batch_size, seq_len, embedding_dim]
        batch_size = tf.shape(inputs)[0]
        query = self.query_dense(inputs)  # (batch_size, seq_len, embed_dim)
        key = self.key_dense(inputs)  # (batch_size, seq_len, embed_dim)
        value = self.value_dense(inputs)  # (batch_size, seq_len, embed_dim)
        query = self.separate_heads(
            query, batch_size
        )  # (batch_size, num_heads, seq_len, projection_dim)
        key = self.separate_heads(
            key, batch_size
        )  # (batch_size, num_heads, seq_len, projection_dim)
        value = self.separate_heads(
            value, batch_size
        )  # (batch_size, num_heads, seq_len, projection_dim)
        attention, weights = self.attention(query, key, value)
        attention = tf.transpose(
            attention, perm=[0, 2, 1, 3]
        )  # (batch_size, seq_len, num_heads, projection_dim)
        concat_attention = tf.reshape(
            attention, (batch_size, -1, self.embed_dim)
        )  # (batch_size, seq_len, embed_dim)
        output = self.combine_heads(
            concat_attention
        )  # (batch_size, seq_len, embed_dim)
        return output


In [30]:
class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = MultiHeadSelfAttention(embed_dim, num_heads)
        self.ffn = keras.Sequential(
            [layers.Dense(ff_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)


In [31]:

class TokenAndPositionEmbedding(layers.Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.token_emb = layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.pos_emb = layers.Embedding(input_dim=maxlen, output_dim=embed_dim)

    def call(self, x):
        maxlen = tf.shape(x)[-1]
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions


In [ ]:
maxlen = 12
embed_dim = 100  # Embedding size for each token
num_heads = 4  # Number of attention heads
ff_dim = 128  # Hidden layer size in feed forward network inside transformer
vocab_size = 12000
inputs = layers.Input(shape=(maxlen,))
embedding_layer = TokenAndPositionEmbedding(maxlen, vocab_size, embed_dim)
x = embedding_layer(inputs)
transformer_block = TransformerBlock(embed_dim, num_heads, ff_dim)
x = transformer_block(x)
x = layers.GlobalAveragePooling1D()(x)
x = layers.Dropout(0.2)(x)
x = layers.Dense(20, activation="relu")(x)
x = layers.Dropout(0.2)(x)
outputs = layers.Dense(8, activation="softmax")(x)

model = keras.Model(inputs=inputs, outputs=outputs)
model.summary()

In [37]:
y_train = train_df['emotion']
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
def label_encode(le, labels):
    enc = le.transform(labels)
    return keras.utils.to_categorical(enc)

def label_decode(le, one_hot_label):
    dec = np.argmax(one_hot_label, axis=1)
    return le.inverse_transform(dec)
y_train = label_encode(label_encoder, y_train)

In [ ]:
x_train = keras.preprocessing.sequence.pad_sequences(trainembed_list, maxlen=maxlen)

### Somehow because of the hardware problem can`t finish traing

In [ ]:
model.compile("adam", "categorical_crossentropy", metrics=["accuracy"])
epochs = 5
batch_size = 128
csv_logger = CSVLogger('D:/Kaggle/dm2020-hw2-nthu/logs/training2_log.csv')
# training!
history = model.fit(x_train, y_train, 
                    epochs=epochs, 
                    batch_size=batch_size, 
                    callbacks=[csv_logger],
                    )
print('training finish')

## TA DNN Method
### using 500 BOW Feature as input

In [35]:

X_train = train_data_BOW_features_500
y_train= train_df['emotion']
X_test = BOW_500.transform(test_df['text'])
y_test = ['joy' for i in range(len(test_df['text']))]
X_train.shape


(1455563, 500)

In [54]:
from collections import Counter
y_trains = Counter(y_train)
y_trains

Counter({'sadness': 193437,
         'disgust': 139101,
         'anticipation': 248935,
         'joy': 516017,
         'trust': 205478,
         'anger': 39867,
         'fear': 63999,
         'surprise': 48729})

In [55]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
def label_encode(le, labels):
    enc = le.transform(labels)
    return keras.utils.to_categorical(enc)

def label_decode(le, one_hot_label):
    dec = np.argmax(one_hot_label, axis=1)
    return le.inverse_transform(dec)

y_train = label_encode(label_encoder, y_train)
y_test = label_encode(label_encoder, y_test)

In [56]:
input_shape = X_train.shape[1]
print('input_shape: ', input_shape)

output_shape = len(label_encoder.classes_)
print('output_shape: ', output_shape)

input_shape:  500
output_shape:  8


In [57]:
from keras.models import Model
from keras.layers import Input, Dense
from keras.layers import ReLU, Softmax

# input layer
model_input = Input(shape=(input_shape, ))  # 500
X = model_input

# 1st hidden layer
X_W1 = Dense(units=64)(X)  # 64
H1 = ReLU()(X_W1)

# 2nd hidden layer
H1_W2 = Dense(units=64)(H1)  # 64
H2 = ReLU()(H1_W2)

# output layer
H2_W3 = Dense(units=output_shape)(H2)  # 4
H3 = Softmax()(H2_W3)

model_output = H3

# create model
model = Model(inputs=[model_input], outputs=[model_output])

# loss function & optimizer
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# show model construction
model.summary()

Model: "functional_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 500)]             0         
_________________________________________________________________
dense_8 (Dense)              (None, 64)                32064     
_________________________________________________________________
re_lu (ReLU)                 (None, 64)                0         
_________________________________________________________________
dense_9 (Dense)              (None, 64)                4160      
_________________________________________________________________
re_lu_1 (ReLU)               (None, 64)                0         
_________________________________________________________________
dense_10 (Dense)             (None, 8)                 520       
_________________________________________________________________
softmax (Softmax)            (None, 8)                

In [62]:
from keras.callbacks import CSVLogger

csv_logger = CSVLogger('D:/Kaggle/dm2020-hw2-nthu/logs/training_log.csv')

# training setting
epochs = 10
batch_size = 128

# training!
history = model.fit(X_train, y_train, 
                    epochs=epochs, 
                    batch_size=batch_size, 
                    callbacks=[csv_logger],
                    )
print('training finish')

Epoch 1/10
11372/11372 [==============================] - 35s 3ms/step - loss: 1.3357 - accuracy: 0.5131
Epoch 2/10
11372/11372 [==============================] - 35s 3ms/step - loss: 1.3341 - accuracy: 0.5137
Epoch 3/10
11372/11372 [==============================] - 36s 3ms/step - loss: 1.3330 - accuracy: 0.5142
Epoch 4/10
11372/11372 [==============================] - 37s 3ms/step - loss: 1.3323 - accuracy: 0.5144
Epoch 5/10
11372/11372 [==============================] - 36s 3ms/step - loss: 1.3317 - accuracy: 0.5142
Epoch 6/10
11372/11372 [==============================] - 36s 3ms/step - loss: 1.3312 - accuracy: 0.5146
Epoch 7/10
11372/11372 [==============================] - 37s 3ms/step - loss: 1.3309 - accuracy: 0.5145
Epoch 8/10
11372/11372 [==============================] - 37s 3ms/step - loss: 1.3306 - accuracy: 0.5148
Epoch 9/10
11372/11372 [==============================] - 37s 3ms/step - loss: 1.3302 - accuracy: 0.5150
Epoch 10/10
11372/11372 [==============================

In [65]:
pred_result = model.predict(X_test, batch_size=128)
pred_result[0:3]

array([[0.02936696, 0.04191859, 0.18879706, 0.10511594, 0.32717097,
        0.16040786, 0.08082838, 0.06639429],
       [0.02026086, 0.05082481, 0.17747216, 0.10967811, 0.2558774 ,
        0.2685597 , 0.04311895, 0.07420798],
       [0.03218394, 0.165647  , 0.3262833 , 0.01006515, 0.20612891,
        0.09517279, 0.02554098, 0.13897792]], dtype=float32)

In [68]:
import numpy as np
pred_result = label_decode(label_encoder, pred_result)
pred_result[:5]

array(['joy', 'sadness', 'disgust', 'joy', 'joy'], dtype=object)

In [70]:
with open('kaggle_demo2.csv','w') as w:
    w.write('id,emotion\n')
    for i,j in zip(test_df['tweet_id'],pred_result):
        w.write(i+','+j+'\n')